In [26]:
%cd /scratch/bruingjde/SNAM2021-code/

from constants import *

/scratch/bruingjde/SNAM2021-code


# Edge temporal features

In [72]:
def logistic_regression(X, y):
  X_train, X_test, y_train, y_test = (
    sklearn.model_selection.train_test_split(X, y))
  pipe = sklearn.pipeline.make_pipeline(
    sklearn.preprocessing.StandardScaler(),
    sklearn.linear_model.LogisticRegression(max_iter=10000, n_jobs=-1))
  pipe.fit(X_train, y_train)
  
  auc = sklearn.metrics.roc_auc_score(
    y_true=y_test, y_score=pipe.predict_proba(X_test)[:,1])
  
  return auc

def get_features() -> dict[FeatureSet, typing.Union[np.ndarray, pd.DataFrame]]:
  aggregation_strategies = ['m0', 'm1', 'q0', 'q25', 'q50', 'q75', 'q100']
  # Get all features that will be used in the logistic_regression.
  return {
    network_index: [
      np.load(
        f'data/{network_index:02}/features/time_edge/'
        f'{topological_feature}_{weighting}_q0.npy'
      )
      for weighting in ['lin', 'exp', 'sqrt']
      for topological_feature in heuristics
    ]
    for network_index in hypergraph_indices
  }

In [73]:
featuresets = get_features()

In [77]:
auc_featuresets = {
  network_index: logistic_regression(
    X=np.array(featureset).T, 
    y=np.load(f'data/{network_index:02}/targets_sampled.npy')
  )
  for network_index, featureset in tqdm(featuresets.items())
}

  0%|          | 0/16 [00:00<?, ?it/s]

In [80]:
pd.Series(auc_featuresets, name='q0').to_pickle('IIb2.pkl')